# Benzinga-Nachrichten-Verarbeitung

## Setup up

### Cluster spin up

In [1]:
use_colab = False
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
    %cd /content/drive/MyDrive/NewsTrading/trading_bot
    %pip install -r requirements_clean.txt

In [2]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [4]:
import os
import re
import pandas as pd
from src.config import config 
from tqdm import tqdm
tqdm.pandas()
# import dask
# import dask.dataframe as dd
# from dask.distributed import Client
# from pyngrok import ngrok, conf

from src.utils.dataframes import parallelize_dataframe
from src.preprocessing.news_parser import infer_author, filter_body

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DEBUG:torchaudio._extension:Failed to initialize sox extension
Traceback (most recent call last):
  File "/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/torchaudio/_extension/__init__.py", line 60, in <module>
    _init_sox()
  File "/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/torchaudio/_extension/utils.py", line 70, in _init_sox
    _load_lib("libtorchaudio_sox")
  File "/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/torchaudio/_extension/utils.py", line 64, in _load_lib
    torch.ops.load_library(path)
  File "/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/torch/_ops.py", line 852, in load_library
    ctypes.CDLL(path)
  File "/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: libsox.so: cannot o

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
## Set Up NGROK-Tunnel
conf.get_default().auth_token = "2WntwErWDt9LxQ2Jfp6C8OxDAMK_7iZVdC1utyZET1PE8cuUg"

public_url = ngrok.connect(8787).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, 8787))

if 'client' not in globals():
  !python -m pip install jupyter-server-proxy
  client = Client()

### System settings

In [ ]:
cwd = os.getcwd()

### CUDF für hardware acceleration

In [ ]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py
# import cudf

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

## Author-Inferenz

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

ddf = dd.read_parquet(input_dir+"*.parquet")
ddf = ddf.repartition(npartitions=50)

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].progress_apply(eval, meta=pd.Series(dtype='object'))

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
%%time
dask.config.set(scheduler="processes")
ddf["inferred_author"] = None
ddf["inferred_author"] = ddf.body.progress_apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

In [ ]:
auhtor_value_counts.sum().diff()

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.apply(lambda x: str(x), meta=pd.Series(dtype="string"))

In [ ]:
ddf.inferred_author.value_counts().compute()

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

In [ ]:
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt. Bzw. generell ist das eine gute Datensäuberungs-Maßnahme. Einerseits verhindern wir,
dass später Aktienkurse einer falschen Aktie zugeordnet wird. Andererseits werden dadurch evtl. auch weniger seriöse Nachrichten herausgefiltert, die nicht
die Kontaktadresszeile des Unternehmens am Ende besitzen, in dem der vollständige Unternehmensname vorkommt.

In [ ]:
ddf = pd.read_parquet(output_dir)
mapper = pd.read_parquet(config.data.benzinga.ticker_name_mapper)

In [ ]:
ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]
ddf["company_name"] = ddf.stocks.map(lambda x: mapper.company_names.loc[x], meta=pd.Series(dtype="string"))
ddf["short_name"] = ddf.stocks.map(lambda x: mapper.short_name.loc[x], meta=pd.Series(dtype="string"))
print(f"Es verbleiben {ddf.shape[0]} Nachrichten, für die wir den Ticker zu einem Firmennamen aufgelösen konnten.")

### Duplikate Entfernen

In [ ]:
samples_before = ddf.shape[0]
ddf = ddf.drop_duplicates()
samples_after = ddf.shape[0]
print(f"{samples_before=}, {samples_after=}")

### Firmennamen-Nachrichtenkörper-Verifikation

In [1]:
def verify_company_name_in_body(df):
    return df.apply(lambda x: bool(re.search(x["short_name"],
                                             x.title + x["body"].replace("( )*\n( )*", " "),
                                             re.IGNORECASE)),
                    axis=1)

In [ ]:
%%time
mask = parallelize_dataframe(ddf, verify_company_name_in_body, n_cores=os.cpu_count())

print(f"Around {len(ddf.stocks.unique())} stocks before filtering and {len(ddf[mask].stocks.unique())} after")

In [ ]:
# If short_name doesn't occurr in the title or in the body, then article seems to be faulty
# mask = ddf.apply(lambda x:
#                  bool(re.search(x["short_name"],
#                                 x.title + x["body"].replace("( )*\n( )*", " "),
#                                 re.IGNORECASE)),
#                  axis=1,
#                  meta=pd.Series(dtype=bool))

In [ ]:
# Filter out faulty news
ddf = ddf[mask]

In [ ]:
# TODO omit if time of this module is not large
# name_function = lambda x: f"data-{x}.parquet"
# ddf.to_parquet(cwd+'/data/bzg/latest2/', name_function=name_function)

Bis zu 5k Nachrichten pro Firma, z.B. AT&T, was in 13 Jahren ca. einer Nachricht pro Tag entspricht. Wir wollen nicht das eine Firma mit vielen Junk-Nachrichten das Modell dominiert. Wobei das Modell hoffentlich dann auch die Junk-Nachrichten als solche erkennt und ignoriert. Eher wichtig noch einen `staleness`-Faktor, also wie ähnlich die Nachricht zu Vorhergegangenen ist (i.e. Nachrichten desselben Tages oder derselben Woche).

Kategorisieren von Nachrichten (mit Text2Topic, wie Salbrechter?) und eliminieren von Business/Strategic etc.
Im Falle von Text2Topic, versuche Estimates des Unternehmens von Dritten zu unterscheiden.

Wichtig!!! Unterscheide zwischen LERN-Phase und PRODUKTIONS-Phase.
Wir können z.B. CLS-Token in der Produktions-Phase vergleichen, in der Lern-Phase aber noch nicht.

Text2Vec -> Business category evtl. entfernen-> Intrastock variance average

## Make reduced ticker name mapping 

In [6]:
ticker_name_mapper = pd.read_parquet("data_shared/ticker_name_mapper.parquet")

ticker_name_mapper_reduced = ddf[["stocks", "company_name", "short_name"]].drop_duplicates(keep="first")
ticker_name_mapper_reduced.to_parquet("data_shared/ticker_name_mapper_reduced.parquet")
print(f"From {ticker_name_mapper.shape[0]} to {ticker_name_mapper_reduced.shape[0]} tickers (reduced)")

11383

## Parsing News Bodies


In [5]:
# ddf = pd.read_parquet('data/latest2')

In [14]:
from src.utils.time import convert_timezone
ddf["time"] = ddf["time"].map(lambda x: convert_timezone(pd.to_datetime(x)))

In [37]:
sample = ddf.iloc[:1000, :]

In [39]:
logging.getLogger().setLevel(logging.WARNING)
def filter_df(df):
    return df.apply(filter_body, axis=1) 

In [40]:
%%time
processed_news: pd.Series = parallelize_dataframe(sample, filter_df, n_cores=os.cpu_count())

/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


CPU times: user 159 ms, sys: 1.21 s, total: 1.37 s
Wall time: 26.7 s


In [43]:
ddf["parsed_body"] = processed_news

In [48]:
ddf.to_parquet(config.data.benzinga.cleaned)

## Filtern von Newstiteln

## Voranstellen von gefilterten Newstiteln an Nachrichtenkörper

# Analyse

## Durschnittlichen Tokenlänge

### Timedeltas zwischen Nachrichtenmeldungen

In [ ]:
import plotly.express as px


In [ ]:
ddf = pd.read_parquet(config.data.benzinga.cleaned)

In [ ]:
ddf = ddf.sort_values("time")
tmp = ddf[["time", "stocks"]]
#### Adding timedeltas to the data frame
news_timedeltas = tmp.groupby("stocks").transform(lambda x: x.diff())
ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))

In [ ]:
news_timedeltas = ddf.timedelta
news_timedeltas.iloc[0].components
timedelta_in_minutes = news_timedeltas.apply(lambda x: x.total_seconds() / 60)
px.histogram(timedelta_in_minutes)